In [ ]:
!pip install fastapi uvicorn pydantic python-dotenv groq



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.3/138.3 kB 2.0 MB/s eta 0:00:00


In [ ]:
!pip -q install groq python-dotenv



In [ ]:
import os, getpass

os.environ["GROQ_API_KEY"] = getpass.getpass("enter api box")


enter api box··········


In [ ]:
from groq import Groq

client = Groq(api_key=os.environ["GROQ_API_KEY"])

# List all currently available models
models = client.models.list().data
model_ids = [m.id for m in models]

print("✅ Available Groq Models:")
for mid in model_ids:
    print("-", mid)
  # you can also use: llama3-8b-8192, mixtral-8x7b-32768


✅ Available Groq Models:
- llama-3.1-8b-instant
- meta-llama/llama-prompt-guard-2-86m
- llama-3.3-70b-versatile
- qwen/qwen3-32b
- meta-llama/llama-prompt-guard-2-22m
- canopylabs/orpheus-arabic-saudi
- whisper-large-v3
- groq/compound-mini
- moonshotai/kimi-k2-instruct
- openai/gpt-oss-120b
- meta-llama/llama-4-scout-17b-16e-instruct
- openai/gpt-oss-20b
- meta-llama/llama-4-maverick-17b-128e-instruct
- whisper-large-v3-turbo
- canopylabs/orpheus-v1-english
- openai/gpt-oss-safeguard-20b
- allam-2-7b
- meta-llama/llama-guard-4-12b
- moonshotai/kimi-k2-instruct-0905
- groq/compound


In [ ]:
PREFERRED_MODELS = [
    # best quality first
    "mixtral-8x7b-32768",
    "llama3-70b-8192",  # old (may not exist)
    "llama-3.1-70b-versatile",  # old (may not exist)

    # fallback fast
    "llama3-8b-8192",
    "gemma2-9b-it"
]

def choose_model():
    available = {m.id for m in client.models.list().data}

    for m in PREFERRED_MODELS:
        if m in available:
            return m

    # if none matched, choose the first available model
    return list(available)[0]

DEFAULT_MODEL = choose_model()
print("✅ Selected model:", DEFAULT_MODEL)


✅ Selected model: moonshotai/kimi-k2-instruct-0905


In [ ]:
def call_groq(prompt: str, model: str = None, temperature: float = 0.3, max_tokens: int = 1024):
    if model is None:
        model = DEFAULT_MODEL

    resp = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ],
        temperature=temperature,
        max_tokens=max_tokens
    )
    return resp.choices[0].message.content


In [ ]:
from dataclasses import dataclass
from typing import List, Dict, Optional


# -------- ROOT PROMPT --------
ROOT_PROMPT = """
SYSTEM (Root Prompt):
You are an expert AI assistant.

Rules:
- Be accurate, structured, and helpful.
- Do not hallucinate. If unsure, say so.
- Use RGC framework (Role, Goal, Context).
- If unclear: use Question Refinement Pattern.
- If important: apply Cognitive Verifier Pattern.
- Think step-by-step internally (do not show hidden chain-of-thought).
- Provide final reasoning as short clear steps.

Output format:
Summary:
Steps:
Final Answer:
""".strip()


# -------- PERSONA --------
@dataclass
class Persona:
    name: str
    expertise: str
    tone: str
    style: str

    def render(self) -> str:
        return f"""
PERSONA:
You are {self.name}.
Expertise: {self.expertise}
Tone: {self.tone}
Style: {self.style}
""".strip()


# -------- RGC --------
def build_rgc(role: str, goal: str, context: str) -> str:
    return f"""
[RGC]
ROLE: {role}
GOAL: {goal}
CONTEXT: {context}
""".strip()


# -------- Prompting Styles --------
def zero_shot(task: str) -> str:
    return f"ZERO-SHOT TASK:\n{task}".strip()


def one_shot(task: str, example_input: str, example_output: str) -> str:
    return f"""
ONE-SHOT TASK:
Example:
Input: {example_input}
Output: {example_output}

Now solve:
{task}
""".strip()


def few_shot(task: str, examples: List[Dict[str, str]]) -> str:
    examples_text = "\n\n".join(
        [f"Example {i+1}:\nInput: {ex['input']}\nOutput: {ex['output']}"
         for i, ex in enumerate(examples)]
    )
    return f"""
FEW-SHOT TASK:
{examples_text}

Now solve:
{task}
""".strip()


# -------- Question Refinement Pattern --------
def question_refinement(user_task: str) -> str:
    return f"""
[QUESTION REFINEMENT PATTERN]
Original request: {user_task}

Do:
1) Rewrite it into a clear single sentence.
2) List missing details.
3) Ask max 2 clarifying questions.
4) Provide a draft response with assumptions.
""".strip()


# -------- Cognitive Verifier Pattern --------
def cognitive_verifier(draft_answer: str) -> str:
    return f"""
[COGNITIVE VERIFIER PATTERN]
Draft answer:
{draft_answer}

Verify:
1) Identify assumptions
2) Check correctness
3) Handle edge cases
4) Improve quality and clarity

Return strictly:
Summary:
Steps:
Final Answer:
""".strip()


# -------- Prompt Engine --------
class PromptEngine:
    def __init__(self, persona: Persona):
        self.persona = persona

    def build_prompt(
        self,
        user_task: str,
        role: str,
        goal: str,
        context: str,
        mode: str = "zero",
        examples: Optional[List[Dict[str, str]]] = None,
        refinement: bool = True
    ) -> str:

        persona_block = self.persona.render()
        rgc_block = build_rgc(role, goal, context)

        if mode == "zero":
            task_block = zero_shot(user_task)
        elif mode == "one":
            if not examples or len(examples) < 1:
                raise ValueError("One-shot requires 1 example")
            ex = examples[0]
            task_block = one_shot(user_task, ex["input"], ex["output"])
        elif mode == "few":
            if not examples or len(examples) < 2:
                raise ValueError("Few-shot requires >=2 examples")
            task_block = few_shot(user_task, examples)
        else:
            raise ValueError("mode must be: zero / one / few")

        refine_block = question_refinement(user_task) if refinement else ""

        full_prompt = f"""
{ROOT_PROMPT}

{persona_block}

{rgc_block}

{task_block}

{refine_block}

Return final output exactly in:
Summary:
Steps:
Final Answer:
""".strip()

        return full_prompt


In [ ]:
def call_groq(prompt: str, model: str = DEFAULT_MODEL, temperature: float = 0.3, max_tokens: int = 1024):
    resp = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ],
        temperature=temperature,
        max_tokens=max_tokens
    )
    return resp.choices[0].message.content


In [ ]:
persona = Persona(
    name="Astra",
    expertise="AI/ML + software engineering + prompt engineering",
    tone="friendly, professional, structured",
    style="mentor-like: explain simple first, then deep with examples"
)

engine = PromptEngine(persona)

def generate_answer(
    user_task: str,
    role: str,
    goal: str,
    context: str,
    mode: str = "zero",
    examples: Optional[List[Dict[str, str]]] = None,
    refinement: bool = True,
    verifier: bool = True,
    model: str = DEFAULT_MODEL
):
    # build prompt
    prompt = engine.build_prompt(
        user_task=user_task,
        role=role,
        goal=goal,
        context=context,
        mode=mode,
        examples=examples,
        refinement=refinement
    )

    # first pass (draft)
    draft = call_groq(prompt, model=model)

    # second pass (verification)
    if verifier:
        verify_prompt = cognitive_verifier(draft)
        final_answer = call_groq(verify_prompt, model=model)
    else:
        final_answer = draft

    return {
        "prompt_used": prompt,
        "draft_answer": draft,
        "final_answer": final_answer
    }


In [ ]:
result = generate_answer(
    user_task="Explain linear search in easy way and provide Java code.",
    role="You are a DSA mentor",
    goal="Teach a beginner clearly with steps and example",
    context="Student knows loops, arrays, and if-else",
    mode="zero",
    refinement=True,
    verifier=True,
    model=DEFAULT_MODEL
)

print("✅ FINAL ANSWER:\n")
print(result["final_answer"])

✅ FINAL ANSWER:

Summary:
Explain how linear search works in plain language and give a ready-to-run Java example.

Steps:
1. Clarify & simplify the request.  
2. List what we still don’t know (none critical for a basic demo).  
3. Supply a concise explanation + code under common assumptions.

Final Answer:
Rewritten request: “Teach a beginner what linear search is and show a simple Java program that looks for a number in an array.”

Missing details (non-blocking):  
- Should the program handle input from the keyboard?  
- Do we need to search other data types (strings, objects)?

Assumptions made:  
- Fixed array of ints, hard-coded for clarity.  
- Search key also hard-coded.  
- Print index if found, “not found” otherwise.

Plain-English explanation:  
Imagine you have a row of numbered boxes. Start at the left, open each box one by one, and stop when you find the number you want. That’s linear search; at worst you check every box.

Java code:
```java
public class LinearSearchDemo {
